In [1]:
# Load các hàm cần thiết
import function
import random

In [2]:
# Định nghĩa lớp ECC
class ECC_sig:
    def __init__(self, a, b, p, P, s, n):
        self.a = a
        self.b = b
        self.p = p
        self.P = P
        self.__s = s
        self.n = n
        self.B = function.mul_k_point(P, s, 0, a, p)
      
    # Khóa công khai
    def public_key(self):
        return (self.a, self.b, self.p, self.P, self.B, self.n)
    
    # Khóa bí mật  
    def private_key(self):
        return self.__s
    
    # Chữ ký
    def sig(self, txt):
        d = self.private_key()
        m = function.hash_map(txt)
        k = random.randint(1, self.n)
        T = function.mul_k_point(self.P, k, 0, self.a, self.p)
        r = T[0] % self.n
        s = (m + d * r) * function.nghichdao(k, self.n) % self.n
        return (r, s)
    
    # Kiểm thử chữ ký
    def ver(self, x, r, s):
        a, b, p, P, B, n = self.public_key()
        m = function.hash_map(x)
        w = function.nghichdao(s, n)
        u1 = m * w % n
        u2 = r * w % n
        u1P = function.mul_k_point(P, u1, 0, a, p)
        u2B = function.mul_k_point(B, u2, 0, a, p)
        T = function.add_point(u1P, u2B, 0, a, p)
        return T[0] % n == r

In [3]:
# Khởi tạo các tham số của thuật toán
# p là số 192 bit
p = 6277101735386680763835789423207666416083908700390324961279
n_point = 6277101735386680763835789423176059013767194773182842284081
a = 6277101735386680763835789423207666416083908700390324961276
b = 2455155546008943817740293915197451784769108058161191238065
P = [602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641]
s = random.randint(1, n_point)
while function.mul_k_point(P, s, 0, a, p) == [0, 0]:
    s = random.randint(0, n_point)

In [4]:
# Khởi tạo ECC với các tham số a, b, p, P, và s
ecc_sig = ECC_sig(a, b, p, P, s, n_point)

# Nhập thông điệp
m = input("Nhập thông điệp: ")

# Kiểm tra xem thông điệp đã được băm có thuộc đường cong elliptic không
while not function.point_in_elliptic(function.hash_map(m), a, b, p):
    print("Thông điệp không hợp lệ trên đường cong elliptic!")
    m = input("Nhập thông điệp: ")

# Mã hóa thông điệp
r, s = ecc_sig.sig(m)

# Ghi thông tin vào file ECC.txt
with open("ECC_sig.txt", "w") as file:
    file.write("Khóa công khai (a, b, p, P, B, n) = " + str(ecc_sig.public_key()) + "\n")
    file.write("Khóa bí mật (s) = " + str(ecc_sig.private_key()) + "\n")
    file.write("Bản rõ m = " + str(m) + "\n")
    file.write("Bản rõ sau khi băm x = " + str(function.hash_map(m)) + "\n")
    file.write("Bản chữ ký (r, s)= " + str(r) + ",\n" + str(s) + "\n")

    file.write("Việc kiểm thử giữa bản tin m và chữ ký (r, s) " + ("thành công\n" if ecc_sig.ver(m, r, s) else "thất bại\n"))

